In [70]:
%load_ext autoreload

import numpy as np
import Z_n_circuit_utils
from Circuits import unitary_sampler
import Circuits.circuit_evolution as evolution
from itertools import combinations
import matplotlib.pyplot as pl
import matplotlib as mpl
import time
import pickle
from colour import Color
from scipy.special import binom
from circuit_utils import *
import Circuits
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
%matplotlib qt5

In [72]:
U = Z_n_circuit_utils.z_n_field(2,1)

In [90]:
L_list = [8,10,12]
N = 5
Kraus_operators = Z_n_circuit_utils.z_n_measurement(N)
g_N = 0.3
p_meas = 0

entropy= {}
error = {}
for L in L_list:
    T = 4*L
    entropy_data = []
    start = time.time()
    for samps in range(200):
        rng = np.random.default_rng(seed=samps)
        # state,_ = Circuits.circuit_evolution.generalized_measurement(state, Kraus_operators, list(range(0,N)),rng)
        # state = get_sharp_initial_state(L,L//2 - (N+1)//2)
        state = np.zeros((2,)*L)
        initial_charge = [0]*L
        initial_charge[:L//2 - (N+1)//2] = [1]*(L//2 - (N+1)//2)
        state[tuple(initial_charge)] = 1

        for _ in range(2*L):
            state = U_1_layer(state,t,L,rng=rng)

        indices_dic = get_indices_dic(L)
        prob_list = [[get_probability(state,L,q,indices=indices_dic[q]) for q in indices_dic]]
        
        for t in range(T):
            state = U_1_layer(state,t,L,rng=rng)
            state,_ = U_1_measurement(state,L,p_meas,rng=rng)
            state = Z_n_layer(state,L,N,g_N)
            prob_list.append([get_probability(state,L,q,indices=indices_dic[q]) for q in indices_dic])

        entropy_data.append([get_entropy(p_list) for p_list in prob_list])
    print(np.array(entropy_data).shape)
    print("L={},time={}".format(L,time.time()-start))
    entropy[L] = np.average(np.array(entropy_data),axis=0)
    error[L] = np.std(np.array(entropy_data),axis=0)/len(entropy_data)**0.5


(200, 33)
L=8,time=4.311682224273682
(200, 41)
L=10,time=7.59628701210022
(200, 49)
L=12,time=28.979996919631958


In [93]:
for L in L_list:
    xdata = np.arange(0,4*L+1,1)/L
    ydata = entropy[L]
    print(ydata.shape)
    err = error[L]
    maximum,_ = max_value(L//2-(N+1)//2,L,N=5)
    print(maximum)
    ydata = ydata/maximum
    err = err/maximum
    pl.errorbar(xdata,ydata,yerr=err,ls='-',marker='o',label=r'$L=$'+str(L))
pl.legend()

(33,)
0.5297061990576545
(41,)
0.5859526183035508
(49,)
0.6172417697303416


In [95]:
list(np.arange(0,12,1).reshape((3,4)))

[array([0, 1, 2, 3]), array([4, 5, 6, 7]), array([ 8,  9, 10, 11])]

qt.qpa.backingstore: Back buffer dpr of 1 doesn't match <_NSViewBackingLayer: 0x2a996b810> contents scale of 2 - updating layer to match.
qt.qpa.backingstore: Back buffer dpr of 1 doesn't match <_NSViewBackingLayer: 0x2a996b810> contents scale of 2 - updating layer to match.
qt.qpa.backingstore: Back buffer dpr of 1 doesn't match <_NSViewBackingLayer: 0x2a996b810> contents scale of 2 - updating layer to match.
qt.qpa.backingstore: Back buffer dpr of 2 doesn't match <_NSViewBackingLayer: 0x2a996b810> contents scale of 1 - updating layer to match.
qt.qpa.backingstore: Back buffer dpr of 1 doesn't match <_NSViewBackingLayer: 0x2a996b810> contents scale of 2 - updating layer to match.


In [557]:
def save_data(N,p_meas):
    samps = 400

    final_data = {}
    filename = 'data/p_meas='+str(p_meas)+'_N='+str(N)
    for g_N in [0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7]:
        final_data[g_N] = {}
        entropy_dic = {}
        error_dic = {}
        full_data_dic = {}
        for L in [6,8,10,12]:
            start = time.time()
            T = 3*L
            entropy_data = []
            for seed_M in range(1,samps+1):
                prob_list,_ = get_data(L,N,T,p_meas,seed_U=1,seed_M=seed_M,seed_N=10*seed_M,g_N=g_N)
                entropy_data.append([get_entropy(p_list) for p_list in prob_list])
            entropy_data = np.array(entropy_data)
            full_data_dic[L] = entropy_data
            assert entropy_data.shape == (samps,T+1)
            entropy_dic[L] = np.average(entropy_data,axis=0)
            error_dic[L] = np.std(entropy_data,axis=0)/(samps-1)**0.5
            print("g_N={},L={},time={}".format(g_N,L,time.time() - start))
        final_data[g_N]['entropy'] = entropy_dic
        final_data[g_N]['error'] = error_dic
        final_data[g_N]['full_data'] = full_data_dic
        with open(filename,'wb') as f:
            pickle.dump(final_data,f)



In [631]:
def max_value(initial_charge,L,N):
    charges = [initial_charge]
    temp = initial_charge + N
    while temp < L+1:
        charges.append(temp)
        temp = temp + N
    temp = initial_charge - N
    while temp >= 0:
        charges.append(temp)
        temp = temp - N
    p_q = []
    for q in charges:
        p_q.append(binom(L,q))
    p_q = np.array(p_q)
    p_q = p_q / np.sum(p_q)
     

    return get_entropy(p_q), sorted(charges)

In [20]:
import pandas as pd

In [58]:
aa = {1:[],2:[],3:[]}
df = pd.DataFrame(aa)
df

,1,2,3


In [59]:
df = pd.concat([df,pd.DataFrame({1:[10],2:[20],3:[None]})],ignore_index=True)
df = pd.concat([df,pd.DataFrame({1:[2,3],2:[3,3],3:[None,3]})],ignore_index=True)

In [60]:
df

,1,2,3
0,10.0,20.0,NaN
1,2.0,3.0,NaN
2,3.0,3.0,3.0


In [63]:
df.loc[len(df)] = [0,-9,8]
len(df)

6

In [69]:
np.array([[[1,2,4],[3,4,4]],[[5,6,4],[7,8,4]],[[5,6,4],[7,8,40]]]).shape

(3, 2, 3)

In [676]:
save_data(N=2,p_meas=0.7)


g_N=0.05,L=6,time=2.688966989517212
g_N=0.05,L=8,time=4.977934837341309
g_N=0.05,L=10,time=9.195031881332397
g_N=0.05,L=12,time=21.0978102684021
g_N=0.1,L=6,time=2.701953172683716
g_N=0.1,L=8,time=5.047152280807495
g_N=0.1,L=10,time=9.451730012893677
g_N=0.1,L=12,time=21.638311862945557
g_N=0.2,L=6,time=2.851486921310425
g_N=0.2,L=8,time=5.306596040725708
g_N=0.2,L=10,time=9.928119897842407
g_N=0.2,L=12,time=22.75848889350891
g_N=0.3,L=6,time=3.0537989139556885
g_N=0.3,L=8,time=5.632601976394653
g_N=0.3,L=10,time=10.642473936080933
g_N=0.3,L=12,time=23.98956322669983
g_N=0.4,L=6,time=3.1025068759918213
g_N=0.4,L=8,time=5.934096813201904
g_N=0.4,L=10,time=10.983277082443237
g_N=0.4,L=12,time=25.193342924118042
g_N=0.5,L=6,time=3.2390100955963135
g_N=0.5,L=8,time=6.0277650356292725
g_N=0.5,L=10,time=11.335202932357788
g_N=0.5,L=12,time=25.994598865509033
g_N=0.6,L=6,time=3.319499969482422
g_N=0.6,L=8,time=6.229722023010254
g_N=0.6,L=10,time=11.805912733078003
g_N=0.6,L=12,time=27.1785759

### Plot vs time for fixed g

In [543]:
def colorFader(c1,c2,mix=0): #fade (linear interpolate) from color c1 (at mix=0) to c2 (mix=1)
    c1=np.array(mpl.colors.to_rgb(c1))
    c2=np.array(mpl.colors.to_rgb(c2))
    return mpl.colors.to_hex((1-mix)*c1 + mix*c2)

In [679]:
g_N_list = [0.1,0.2,0.7]
N = 2
window = 1
p_meas = 0.7
pl.figure(1,figsize=(8,7))
g_colors = [colorFader('red','green',mix=x) for x in np.linspace(0,1,len(g_N_list))]
filename = 'data/p_meas='+str(p_meas)+'_N='+str(N)
with open(filename,'rb') as f:
    final_data = pickle.load(f)

for i,g_N in enumerate(g_N_list):
    entropy_dic = final_data[g_N]['entropy']
    error_dic = final_data[g_N]['error']
    Ldata = np.array(list(sorted(entropy_dic.keys())))
    alpha_L = np.linspace(0.2,1,len(Ldata))
    for j,L in enumerate(Ldata):
        xdata = np.arange(0,3*L+1,1)/L
        maximum,_ = max_value(L//2-N//2,L,N)
        ydata = entropy_dic[L].copy()/maximum
        err_data = error_dic[L].copy()/maximum
        ydata,err_data = rolling_average(ydata,err_data,window)
        # ydata = ydata/ydata[0]
        # err_data = err_data/ydata[0]
        pl.errorbar(xdata,ydata,yerr=err_data,ls='-',color=g_colors[i],alpha=alpha_L[j],marker='o',label='L={},g={}'.format(L,g_N))
pl.ylabel(r'$\frac{\rm{Entropy}}{\rm{Maximum Value}}$',fontsize =20)
pl.xlabel(r'$t/L$')
pl.legend(ncol=len(g_N_list))
pl.tight_layout()

In [802]:
N = 4
p_meas = 0.2
filename = 'data/p_meas='+str(p_meas)+'_N='+str(N)
with open(filename,'rb') as f:
    final_data = pickle.load(f)
window = 1

pl.figure(2,figsize=(8,7))
g_N_list = list(sorted(final_data.keys()))
xdata = {}
ydata = {}
errdata = {}
for g_N in g_N_list:
    entropy_dic = final_data[g_N]['entropy']
    error_dic = final_data[g_N]['error']
    Ldata = np.array(list(sorted(entropy_dic.keys())))
    for L in Ldata:
        if L not in ydata:
            ydata[L] = []
            errdata[L] = []
        maximum,_ = max_value(L//2-N//2,L,N)
        ydata[L].append(np.average(entropy_dic[L][-6:]).copy()/maximum)
        errdata[L].append(np.average(error_dic[L][-6:].copy())/maximum)

for L in ydata:
    pl.errorbar(g_N_list,ydata[L],yerr=errdata[L],ls='-',marker='o',label='L={}'.format(L,g_N))
pl.ylabel(r'$\frac{\rm{Entropy}}{\rm{Maximum Value}}$',fontsize =20)
pl.xlabel(r'$g$')
pl.legend(ncol=len(g_N_list))
pl.tight_layout()


In [99]:
[get_entropy(p_list) for p_list in a]

[0.6086458979772175,
 0.6086458979772174,
 0.6086458979772174,
 0.6086458979772171,
 0.6086458979772174,
 0.6086458979772177,
 0.6086458979772177,
 0.6086458979772178,
 0.6086458979772178,
 0.6086458979772177,
 2.220446049250313e-16,
 1.1102230246251564e-16,
 -2.220446049250313e-16,
 -0.0,
 -0.0,
 1.1102230246251564e-16,
 -0.0,
 -2.220446049250313e-16,
 -0.0,
 7.771561172376093e-16,
 4.440892098500625e-16,
 1.110223024625156e-15,
 1.332267629550187e-15,
 -0.0,
 -0.0,
 6.661338147750936e-16,
 8.881784197001248e-16,
 -0.0,
 -2.220446049250313e-16,
 -0.0,
 -2.220446049250313e-16,
 -0.0,
 -0.0,
 2.220446049250313e-16,
 4.440892098500625e-16,
 -2.220446049250313e-16,
 -6.661338147750941e-16,
 -8.881784197001256e-16,
 -8.881784197001256e-16,
 -0.0,
 -0.0,
 4.440892098500625e-16,
 5.551115123125782e-16,
 5.551115123125782e-16,
 6.661338147750936e-16,
 1.110223024625156e-15,
 2.220446049250313e-16,
 -2.220446049250313e-16,
 -4.440892098500627e-16,
 -0.0,
 -0.0,
 -2.220446049250313e-16,
 1.1102

In [789]:
from Circuits import unitary_sampler
import Circuits.circuit_evolution as evolution
L = 14
T = 2*L
N = 5
for _ in range(400):
    state = np.ones((2,)*L)
    state = state/np.sum(np.abs(state)**2)**0.5 # normalize the state
    seed = np.random.randint(1,1000000000)
    rng = np.random.default_rng(seed)
    for t in range(T):
        U_list = [unitary_sampler.U_1_sym_gate_sampler(rng) for _ in range(L//2)]
        if t%2 == 0:
            state = evolution.evenlayer(state,U_list,L)
        else:
            state = evolution.oddlayer(state,U_list,L,BC='PBC')
        state,_ = evolution.measurement_layer(state,np.arange(0,7,1),np.random.default_rng(1))
        state = circuit_utils.Z_n_layer(state,L,N,g_N=0.5,rng=np.random.default_rng(1))

In [801]:
aa = np.empty((1,2))
xx = np.arange(0,16,1).reshape((2,8))
zz = np.zeros((4,2))
list(xx)

[array([0, 1, 2, 3, 4, 5, 6, 7]), array([ 8,  9, 10, 11, 12, 13, 14, 15])]

In [100]:
def aa(i):
    return i+1
def aa2(i):
    return i+2


In [103]:
temp = aa2
temp(4)

6

qt.qpa.backingstore: Back buffer dpr of 1 doesn't match <_NSViewBackingLayer: 0x2a996b810> contents scale of 2 - updating layer to match.
qt.qpa.backingstore: Back buffer dpr of 1 doesn't match <_NSViewBackingLayer: 0x2a996b810> contents scale of 2 - updating layer to match.
qt.qpa.backingstore: Back buffer dpr of 1 doesn't match <_NSViewBackingLayer: 0x2a996b810> contents scale of 2 - updating layer to match.
qt.qpa.backingstore: Back buffer dpr of 2 doesn't match <_NSViewBackingLayer: 0x2a996b810> contents scale of 1 - updating layer to match.
qt.qpa.backingstore: Back buffer dpr of 1 doesn't match <_NSViewBackingLayer: 0x2a996b810> contents scale of 2 - updating layer to match.
qt.qpa.backingstore: Back buffer dpr of 1 doesn't match <_NSViewBackingLayer: 0x2a996b810> contents scale of 2 - updating layer to match.
qt.qpa.backingstore: Back buffer dpr of 2 doesn't match <_NSViewBackingLayer: 0x2a996b810> contents scale of 1 - updating layer to match.
qt.qpa.backingstore: Back buffer d